In [80]:
import requests
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import plotly.express as px

In [ ]:
def get_page(vacancy, page=0):
    """
    Создаем метод для получения страницы со списком вакансий.
    Аргументы:
        page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
    """
    # Справочник для параметров GET-запроса
    params = {
        'text': f'NAME:{vacancy}', # Текст фильтра. В имени должно быть слово "Аналитик"
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }
    
    
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    # data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return req.json()

In [ ]:
res = []
def get_vacancy(vacancy, pages=10):
    for page in tqdm(range(pages)):
        req = get_page(vacancy, page)
        res.extend(req['items'])

for vacancy in ['аналитик', 'данные', 'data', 'analyst']:
    get_vacancy(vacancy)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [121]:
df = pd.DataFrame(res)
df.shape

(3010, 30)

In [123]:
columns = ['name', 'salary', 'schedule', 'area']
df = df[columns]
df.head()

,name,salary,schedule,area
0,Аналитик (Junior),"{'from': 100000, 'to': None, 'currency': 'RUR'...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '1', 'name': 'Москва', 'url': 'https://..."
1,Аналитик,"{'from': 80000, 'to': None, 'currency': 'RUR',...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '1', 'name': 'Москва', 'url': 'https://..."
2,Системный аналитик,"{'from': 250000, 'to': 350000, 'currency': 'RU...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '1', 'name': 'Москва', 'url': 'https://..."
3,Амбициозный Дата аналитик,None,"{'id': 'fullDay', 'name': 'Полный день'}","{'id': '2759', 'name': 'Ташкент', 'url': 'http..."
4,Аналитик бизнес-процессов,"{'from': 200000, 'to': 300000, 'currency': 'RU...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '2', 'name': 'Санкт-Петербург', 'url': ..."


In [124]:
df["city"] = (df["area"]
                     .apply(lambda x: x.get("name") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["schedule"] = (df["schedule"]
                     .apply(lambda x: x.get("id") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_from"] = (df["salary"]
                     .apply(lambda x: x.get("from") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_to"] = (df["salary"]
                     .apply(lambda x: x.get("to") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_currency"] = (df["salary"]
                     .apply(lambda x: x.get("currency") 
                                      if isinstance(x, dict) 
                                      else np.nan))

<ipython-input-124-3ba0d35b117a>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-124-3ba0d35b117a>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-124-3ba0d35b117a>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-124-3ba

In [125]:
df.head()

,name,salary,schedule,area,city,salary_from,salary_to,salary_currency
0,Аналитик (Junior),"{'from': 100000, 'to': None, 'currency': 'RUR'...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",Москва,100000.0,NaN,RUR
1,Аналитик,"{'from': 80000, 'to': None, 'currency': 'RUR',...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",Москва,80000.0,NaN,RUR
2,Системный аналитик,"{'from': 250000, 'to': 350000, 'currency': 'RU...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",Москва,250000.0,350000.0,RUR
3,Амбициозный Дата аналитик,None,fullDay,"{'id': '2759', 'name': 'Ташкент', 'url': 'http...",Ташкент,NaN,NaN,NaN
4,Аналитик бизнес-процессов,"{'from': 200000, 'to': 300000, 'currency': 'RU...",fullDay,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...",Санкт-Петербург,200000.0,300000.0,RUR


In [126]:
df.tail()

,name,salary,schedule,area,city,salary_from,salary_to,salary_currency
3005,Бизнес-аналитик,None,remote,"{'id': '90', 'name': 'Томск', 'url': 'https://...",Томск,NaN,NaN,NaN
3006,Системный аналитик процесса регулярной отчётности,None,fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",Москва,NaN,NaN,NaN
3007,Project manager/Business Analyst,None,fullDay,"{'id': '1002', 'name': 'Минск', 'url': 'https:...",Минск,NaN,NaN,NaN
3008,Ведущий бизнес аналитик IT (senior/middle),"{'from': 180000, 'to': 350000, 'currency': 'RU...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",Москва,180000.0,350000.0,RUR
3009,Ведущий системный аналитик (senior/middle),"{'from': 200000, 'to': 400000, 'currency': 'RU...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",Москва,200000.0,400000.0,RUR


In [129]:
df['salary_currency'].value_counts()

RUR    673
USD     40
KZT     21
EUR     12
UZS      6
BYR      4
AZN      1
Name: salary_currency, dtype: int64

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3010 entries, 0 to 3009
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             3010 non-null   object 
 1   salary           757 non-null    object 
 2   schedule         3010 non-null   object 
 3   area             3010 non-null   object 
 4   city             3010 non-null   object 
 5   salary_from      631 non-null    float64
 6   salary_to        484 non-null    float64
 7   salary_currency  757 non-null    object 
dtypes: float64(2), object(6)
memory usage: 188.2+ KB


In [ ]:
df = df[df['salary_currency'] == 'RUR'][['city', 'salary_from', 'salary_to']]

In [143]:
df

,city,salary_from,salary_to
0,Москва,100000.0,NaN
1,Москва,80000.0,NaN
2,Москва,250000.0,350000.0
4,Санкт-Петербург,200000.0,300000.0
5,Самара,52200.0,52200.0
...,...,...,...
2989,Новосибирск,50000.0,NaN
2991,Ульяновск,60000.0,100000.0
2996,Москва,150000.0,150000.0
3008,Москва,180000.0,350000.0
